Я сделал target.csv (Count, IsTrain, IsTest, IsValidation) индекс совпадает с date.csv, пофиксил твою функцию с вычеслением глобальной нумерации недель, два твоих бага пофиксил - простые(видимо ты их случайно заккомитил), со сборкой средних у меня возникли запары, сам делай. Учитывай, что по трейну нужно считать признаки не только для самого трейна, но и для тестовой и валидационной выборки (вот с этим у меня проблемы возникли)

In [3]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation

In [2]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort(['Date'])

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [3]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [4]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [5]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())

In [6]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [7]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [8]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days  - start_date.weekday() - 1 ) / 7))
#теперь нет смещения, это могло вводить неоднозначноть, т.е. одной недели могли противоречить две фичи

In [9]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [10]:
date.to_csv('data_transform/date.csv', index=None)

In [11]:
date.head()

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,278,0,0,0
1,2011-10-07,2011,10,7,4,279,0,0,1
2,2011-10-09,2011,10,9,6,281,0,0,3
3,2011-11-15,2011,11,15,1,318,1,5,40
4,2011-12-08,2011,12,8,3,341,2,8,63


# target.csv

In [4]:
sizeValidationSample = 0.15

In [5]:
target = pd.read_csv('data_original/common.csv')[['Count']]
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [6]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [7]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [8]:
target['IsTrain'] = target.IsTrain.apply(lambda x: 1 if ((x in indexTrain) & (target.IsTest[x] == 0)) else 0)
target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsTest[x] == 0)) else 0)

In [9]:
target.to_csv('data_transform/target.csv', index = None)

In [10]:
target.head(10)

,Count,IsTest,IsTrain,IsValidation
0,8.0,0,1,0
1,88.0,0,1,0
2,1.0,0,1,0
3,1.0,0,1,0
4,2.0,0,1,0
5,43.0,0,1,0
6,10.0,0,1,0
7,1.0,0,1,0
8,60.0,0,1,0
9,NaN,1,0,0


# city.csv


In [18]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [19]:
city.head()

,Point,CityName,CityID,BranchNumber
0,Сыктывкар-1,Сыктывкар,0,1
1,Сыктывкар-2,Сыктывкар,0,2
2,Ухта-1,Ухта,1,1
3,Смоленск-1,Смоленск,2,1
4,Сарапул-1,Сарапул,3,1


# internal_features.csv

In [7]:
src = pd.read_csv('data_transform/date.csv')
src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']]
dst = pd.DataFrame() 

In [2]:
tmp = pd.read_csv('data_transform/target.csv')
tmp = pd.read_csv('data_transform/target.csv')[tmp.IsTrain == True]

trainIndex = tmp.index

In [3]:
src = pd.read_csv('data_transform/date.csv').iloc[trainIndex]
src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']].iloc[trainIndex]
dst = pd.DataFrame() 

In [4]:
src.head(10)

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count,Point
0,2011-10-06,2011,10,6,3,278,0,0,0,8.0,Сыктывкар-1
1,2011-10-07,2011,10,7,4,279,0,0,1,88.0,Сыктывкар-1
2,2011-10-09,2011,10,9,6,281,0,0,3,1.0,Сыктывкар-1
4,2011-12-08,2011,12,8,3,341,2,8,63,2.0,Сыктывкар-1
5,2011-12-09,2011,12,9,4,342,2,8,64,43.0,Сыктывкар-1
7,2011-12-11,2011,12,11,6,344,2,8,66,1.0,Сыктывкар-1
8,2011-12-13,2011,12,13,1,346,2,9,68,60.0,Сыктывкар-1
10,2011-12-15,2011,12,15,3,348,2,9,70,96.0,Сыктывкар-1
12,2011-12-17,2011,12,17,5,350,2,9,72,112.0,Сыктывкар-1
13,2011-12-18,2011,12,18,6,351,2,9,73,100.0,Сыктывкар-1


## Mean

### by date independed from points

In [5]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [6]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [7]:
dst.head()

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,112.033104,104.223060,8.0,32.333333,32.333333,91.475701,88.631579,96.387097
1,112.033104,106.942082,88.0,32.333333,32.333333,123.167972,88.631579,88.774194
2,112.033104,108.045872,1.0,32.333333,32.333333,123.488522,88.631579,149.000000
4,106.917219,110.747041,2.0,99.187500,15.333333,91.475701,88.631579,75.542169
5,106.917219,108.045872,43.0,99.187500,15.333333,123.167972,88.631579,79.329670


### by date depended from points

In [8]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)

In [9]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [10]:
dst.to_csv('data_transform/internal_features.csv')

In [11]:
dst.head()

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,112.033104,104.223060,8.0,32.333333,32.333333,91.475701,88.631579,96.387097,263.472222,230.933333,8.0,32.333333,32.333333,214.486034,88.631579,190.000000
1,112.033104,106.942082,88.0,32.333333,32.333333,123.167972,88.631579,88.774194,263.472222,261.116279,88.0,32.333333,32.333333,286.132275,88.631579,179.666667
2,112.033104,108.045872,1.0,32.333333,32.333333,123.488522,88.631579,149.000000,263.472222,247.545455,1.0,32.333333,32.333333,264.702128,88.631579,222.000000
4,106.917219,110.747041,2.0,99.187500,15.333333,91.475701,88.631579,75.542169,236.954545,251.577778,2.0,99.187500,15.333333,214.486034,88.631579,182.600000
5,106.917219,108.045872,43.0,99.187500,15.333333,123.167972,88.631579,79.329670,236.954545,247.545455,43.0,99.187500,15.333333,286.132275,88.631579,183.800000


# All in one

In [19]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv', index_col = 0)

need_columns = date.columns
data[need_columns] = date[need_columns]

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

In [22]:
data.head(20)

,Count,IsTest,IsTrain,IsValidation,Date,Year,Month,MonthDay,WeekDay,YearDay,...,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8.0,0,1,0,2011-10-06,2011,10,6,3,278,...,88.631579,96.387097,263.472222,230.933333,8.0,32.333333,32.333333,214.486034,88.631579,190.000000
1,88.0,0,1,0,2011-10-07,2011,10,7,4,279,...,88.631579,88.774194,263.472222,261.116279,88.0,32.333333,32.333333,286.132275,88.631579,179.666667
2,1.0,0,1,0,2011-10-09,2011,10,9,6,281,...,88.631579,149.000000,263.472222,247.545455,1.0,32.333333,32.333333,264.702128,88.631579,222.000000
3,1.0,0,0,1,2011-11-15,2011,11,15,1,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,0,1,0,2011-12-08,2011,12,8,3,341,...,88.631579,75.542169,236.954545,251.577778,2.0,99.187500,15.333333,214.486034,88.631579,182.600000
5,43.0,0,1,0,2011-12-09,2011,12,9,4,342,...,88.631579,79.329670,236.954545,247.545455,43.0,99.187500,15.333333,286.132275,88.631579,183.800000
6,10.0,0,0,1,2011-12-10,2011,12,10,5,343,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,0,1,0,2011-12-11,2011,12,11,6,344,...,88.631579,102.960526,236.954545,221.205882,1.0,99.187500,15.333333,264.702128,88.631579,139.000000
8,60.0,0,1,0,2011-12-13,2011,12,13,1,346,...,88.631579,133.021978,236.954545,221.244898,60.0,99.187500,92.000000,200.833333,88.631579,214.000000
9,NaN,1,0,0,2011-12-14,2011,12,14,2,347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data.to_csv('data_transform/all.csv')

In [14]:
test = pd.read_csv('data_transform/all.csv', index_col=0)